In [ ]:
import iris
import os
import glob
import datetime
from iris.experimental.equalise_cubes import equalise_attributes
import numpy as np
import matplotlib.pyplot as plt
import iris.quickplot as qplt

In [ ]:
data_path = 'ftp.pa.op.dlr.de/pub/righi/BACKEND-DATA/ETHZ_CMIP5/historical/Amon'
ps_cube_list = iris.load(glob.glob(os.path.join(data_path, "ps/GFDL-ESM2G/r1i1p1/ps_Amon_GFDL-ESM2G_historical_r1i1p1_*")))
tro3_cube = iris.load_cube(os.path.join(data_path, "tro3/GFDL-ESM2G/r1i1p1/tro3_Amon_GFDL-ESM2G_historical_r1i1p1_200001-200212.nc"))

for cube in ps_cube_list:
    cube.attributes['history'] = None
    
equalise_attributes(ps_cube_list)
ps_cube = ps_cube_list.concatenate_cube()
print ps_cube.shape
print tro3_cube.shape
print tro3_cube.units

![Ozone vs height](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Ozone_altitude_UV_graph.svg/300px-Ozone_altitude_UV_graph.svg.png)

![Pressure vs Height](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Pressure_air.svg/300px-Pressure_air.svg.png)

# Pressure layers taking surface pressure into account

In [ ]:
import toz
reload(toz)
bounds_widths = toz.pressure_level_widths
create_pressure_cube = toz.create_pressure_cube
apply_bounds_widths = toz.apply_bounds_widths
pressure_layer_widths = toz.pressure_layer_widths
total_column_ozone = toz.total_column_ozone

In [ ]:
# test create_pressure_cube
top_limit=100
p_4d = create_pressure_cube(tro3_cube, ps_cube, top_limit)
print (p_4d[:, 0, :, :] == ps_cube.data).all()
print (p_4d[:, -1, :, :] == top_limit).all()

In [ ]:
# test apply_bounds_width
top_limit=100
p_4d = create_pressure_cube(tro3_cube, ps_cube, top_limit)

%time layer_thicknesses = apply_bounds_widths(p_4d)

In [ ]:
# test apply_bounds_width
assert layer_thicknesses.shape == tro3_cube.shape
for i in range(layer_thicknesses.shape[1]):
    plt.contourf(layer_thicknesses[0, i,:, :], cmap='plasma')

    # Add coastlines to the map
    plt.colorbar()
    plt.show()

In [ ]:
#tro3_cube = tro3_cube[0:2, :, :, :]
#ps_cube = ps_cube[0:2, :, :]

ozone_col_cube = total_column_ozone(tro3_cube, ps_cube)
print ozone_col_cube

qplt.contourf(ozone_col_cube[0,:,:])
plt.gca().coastlines()
plt.title('Total column ozone')
plt.show()

In [ ]:
const_ps_cube = ps_cube.copy()
const_ps_cube.data = np.ones(shape=const_ps_cube.data.shape) * 102000

ozone_col_cube_const_p = total_column_ozone(tro3_cube, const_ps_cube)
print ozone_col_cube

qplt.contourf(ozone_col_cube[0,:,:])
plt.gca().coastlines()
plt.title('Total column ozone')
plt.show()

In [ ]:
qplt.contourf(ozone_col_cube[0,:,:] - ozone_col_cube_const_p[0,:,:])
plt.gca().coastlines()
plt.title('Total column ozone DIFFERENCE')
plt.show()